Here's my attempt to solve that problem. Below are the imports, as well as some dictionaries that will be used later. Might help to look at some definitions in my proposal if yu're not clear on anything.

In [1]:
import pandas as pd
import numpy as np
recession1_years = [ '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007']
recession2_years = [ '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']
schema_dict = { 'area_fips':str,  'own_code':str,  'industry_code':str,  'agglvl_code':str,  'size_code':str,  'year':int,  'qtr':int,  'disclosure_code':str,  'area_title':str,  'own_title':str,  'industry_title':str,  'agglvl_title':str,  'size_title':str,  'qtrly_estabs':int,  'month1_emplvl':int,  'month2_emplvl':int,  'month3_emplvl':int,  'total_qtrly_wages':int,  'taxable_qtrly_wages':int,  'qtrly_contributions':int,  'avg_wkly_wage':int,  'lq_disclosure_code':str,  'lq_qtrly_estabs':float,  'lq_month1_emplvl':float,  'lq_month1_emplv2':float,  'lq_month1_emplv3':float,  'lq_total_qtrly_wages':float,  'lq_taxable_qtrly_wages':float,  'lq_qrtly_contributions':float,  'oty_disclosure_code':str,  'oty_qtrly_estabs':int,  'oty_qtrly_estabs_pct_chg':float,  'oty_month1_emplvl_chg':int,  'oty_month1_emplvl_pct_chg':float,  'oty_month2_emplv_chg':int,  'oty_month2_emplvl_pct_chg':float,  'oty_month3_emplvl_chg':int,  'oty_month3_emplvl_pct_chg':float,  'oty_total_qtrly_wages_chg':int,  'oty_total_qtrly_wages_pct_chg':float,  'oty_taxable_qtrly_wages_chg':int,  'oty_taxable_qtrly_wages_pct_chg':float,  'oty_qrtly_contributions_chg':int,  'oty_qrtly_contributions_pct_chg':float,  'oty_avg_wkly_wage_chg':int,  'oty_avg_wkly_wage_pct_chg':float}  
drop_columns = ['industry_code',  'own_code',  'size_code',  'disclosure_code',  'industry_title',  'own_title',  'size_title',  'lq_disclosure_code',  'oty_disclosure_code',  'oty_month1_emplvl_chg',  'oty_month2_emplvl_chg',  'oty_month3_emplvl_chg',  'oty_total_qtrly_wages_chg',  'oty_taxable_qtrly_wages_chg',  'oty_qtrly_contributions_chg',  'oty_avg_wkly_wage_chg',  'lq_qtrly_estabs_count',  'lq_month1_emplvl',  'lq_month2_emplvl',  'lq_month3_emplvl',  'lq_total_qtrly_wages',  'lq_taxable_qtrly_wages',  'lq_qtrly_contributions',  'oty_qtrly_estabs_count_chg',  'oty_qtrly_estabs_count_pct_chg',  'oty_month1_emplvl_pct',  'oty_month2_emplvl_pct',  'oty_month3_emplvl_pct',  'oty_total_qtrly_wages_pct',  'oty_taxable_qtrly_wages_chg',  'oty_qtrly_contributions_pct',  'oty_avg_wkly_wage_pct',  'oty_taxable_qtrly_wages_chg.1',  'lq_avg_wkly_wage',  'taxable_qtrly_wages',  'qtrly_contributions']
import os
os.chdir('/home/cj/Documents/dsi/capstones/report-card-recession')


Functions to refer to.

In [2]:
def add_qtrid(df):
    '''
    adds a column for the year and quarter.

    params: df(dataframe)
    returns: dataframe with column added'''
    df['qtrid'] = df['year'] + (df['qtr']/4)
    return df

def import_one(year):
    '''brings a single year's woth of data into a dataframe. Used for initial EDA. 
    Referenced in import_all

    params: year(str)
    returns: df(dataframe)'''
    filepath = '../data/' + str(year) + '.csv'
    #all relevant csvs are renamed with only the year
    df = pd.read_csv(filepath, dtype = schema_dict)
    #schema_dict is found in dictionaries.py
    for column in drop_columns:
        if column in df.columns:
            df = df.drop([column], axis = 1)
    return df

def import_all(years):
    '''combines as many years ofdata into a single dataframe, as well as adding quater id
    References import_one and add_qtrid

    params: years (list of str)
    returns: df (dataframe)'''
    df = import_one(years[0])
    for year in years[1:]:
        df = df.append(import_one(year))
    df = add_qtrid(df)
    return df

Create the initial dataframe with all eight years of data appended.

In [ ]:
df = import_all(recession2_years)

Do a pivotable. Will give you two columns (str) for the area in question, and one for *each* quarter of data. Total job numbers only.

In [3]:
def create_df_empl(df):
    return df.pivot_table(columns = 'qtrid', values = 'month3_emplvl', index = ['area_fips', 'area_title'], aggfunc = np.sum)



Here's the part that works. the timeline dataframe is made, the nadir is calculated as the minimum of the entire timeline, and the quarter in whic it happens is saved as nadir_qtr

In [4]:
def calc_nadir(s):
    assert isinstance(s, pd.Series)
    return s.min()

def calc_nadir_qtr(s):
    return s.argmin()

def calc_pre_peak(s):
    return s[ : s.argmin()].max()

def calc_pre_peak_quarter(s):
    try:
        qtr = s[ : s.argmin()].argmax()
    except:
        qtr = None
    return qtr

def calc_post_peak(s):
    return s[s.argmin() : ].max()

def calc_post_peak_qtr(s):
    return s[s.argmin() : ].argmax() + s.argmin()

def calc_nadir(s):
    assert isinstance(s, pd.Series)
    return s.min()

def calc_nadir_qtr(s):
    return s.argmin()

def calc_pre_peak(s):
    return s[ : s.argmin()].max()

def calc_pre_peak_quarter(s):
    try:
        qtr = s[ : s.argmin()].argmax()
    except:
        qtr = None
    return qtr

def calc_post_peak(s):
    return s[s.argmin() : ].max()

def calc_post_peak_qtr(s):
    return s[s.argmin() : ].argmax() + s.argmin()

In [11]:
df = import_all(recession1_years)
df = df.pivot_table(columns = 'qtrid', values = 'month3_emplvl', index = ['area_fips', 'area_title'], aggfunc = np.sum)
df = df.reset_index()
df2 = df.drop(columns = ['area_fips', 'area_title'])
df2 = df2.reset_index()
df2 = df2.drop(columns = 'index')



In [12]:
#this specifies when the jobs numbers "bottom-out" during the recession
# df = df.drop(columns = ['area_fips', 'area_title'])
df2= df2.fillna(0)
df2.info()

nadir = df2.apply(lambda x: calc_nadir(x), axis=1).rename('nadir')
#counts the number of quarters to the nadir since the beginning of the timeframe
nadir_qtr = df2.apply(lambda x: calc_nadir_qtr(x), axis=1).rename('nadir_qtr')
    
#computes the highest points before and after the nadir, and captures the quarter count
pre_peak = df2.apply(lambda x: calc_pre_peak(x), axis=1).rename('pre_peak')
pre_peak_qtr = df2.apply(lambda x: calc_pre_peak_quarter(x), axis=1).rename('pre_peak_qtr')
post_peak = df2.apply(lambda x: calc_post_peak(x), axis=1).rename('post_peak')
post_peak_qtr = df2.apply(lambda x: calc_post_peak_qtr(x), axis=1).rename('post_peak_qtr')
    
#puts the computed points in a dataframe, joins with timeline
df_results = pd.concat([df['area_fips'], nadir, nadir_qtr, pre_peak, pre_peak_qtr, post_peak, post_peak_qtr], axis=1)
df = df.join(df_results, how = 'outer', rsuffix = '_derive')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4403 entries, 0 to 4402
Data columns (total 32 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   2000.25  4403 non-null   float64
 1   2000.5   4403 non-null   float64
 2   2000.75  4403 non-null   float64
 3   2001.0   4403 non-null   float64
 4   2001.25  4403 non-null   float64
 5   2001.5   4403 non-null   float64
 6   2001.75  4403 non-null   float64
 7   2002.0   4403 non-null   float64
 8   2002.25  4403 non-null   float64
 9   2002.5   4403 non-null   float64
 10  2002.75  4403 non-null   float64
 11  2003.0   4403 non-null   float64
 12  2003.25  4403 non-null   float64
 13  2003.5   4403 non-null   float64
 14  2003.75  4403 non-null   float64
 15  2004.0   4403 non-null   float64
 16  2004.25  4403 non-null   float64
 17  2004.5   4403 non-null   float64
 18  2004.75  4403 non-null   float64
 19  2005.0   4403 non-null   float64
 20  2005.25  4403 non-null   float64
 21  2005.5   4403 

In [17]:
df['recovery'] = (df['post_peak'] >= df['pre_peak']) *1
df['full_decline'] = (df['nadir_qtr'] == 32) * -1

In [18]:
pd.crosstab(df['recovery'], df['full_decline'])


full_decline,0
recovery,
0,2435
1,1968


In [ ]:
    df_empl = df_empl.drop(columns=[2007.25, 2007.5, 2007.75, 2008.0,
       2008.25, 2008.5, 2008.75, 2009.0, 2009.25, 2009.5, 2009.75, 2010.0, 2010.25, 2010.5, 2010.75, 2011.0, 2011.25, 2011.5, 2011.75,2012.0, 2012.25, 2012.5, 2012.75, 2013.0, 2013.25, 2013.5, 2013.75, 2014.0, 2014.25, 2014.5, 2014.75, 2015.0, 2015.25, 2015.5,
       2015.75, 2016.0, 2016.25, 2016.5, 2016.75, 2017.0, 2017.25, 2017.5, 2017.75, 2018.0, 2018.25, 2018.5, 2018.75, 2019.0, 2019.25, 2019.5, 2019.75, 2020.0], axis =1)

In [ ]:
df_empl['nadir_qtr'].value_counts()

Here is a *very* messy attempt to brute force it. Do not feel obligated to preserve any of this. It doesn't work (at all) but it does give an idea of what I'm trying to accomplish here.


In [ ]:
#instantiates the five columns I will be making from the nasty for loop
df_empl['pre-peak'] = 0
df_empl['pre-peak_qtr'] = 0
df_empl['post-peak'] = 0
df_empl['post-peak_qtr'] = 0
df_empl['recov_qtr'] = 0

#iterates through each row of the dataframe
for index, row in df_empl.iterrows():
    #save the nadir value in a variable
    nadir = row['nadir']
    #need an iterator and a var to store where the slice needs to happen on each row
    counter = 0
    slicer = 0
    #make a np array from the row
    row_array = np.array(row)
    #iterate through each value in the array
    for num in row_array:
        #check each value to see if it is the same as the nadir. If so, set the slicer var.
        if num == nadir:
            slicer = counter
            break
        else:
            counter += 1
    
    #break the array into two peices, before and after the nadir
    pre_array = row_array[2:slicer]
    post_array = row_array[slicer +1:]

    #set local vars for the pre and post peaks, and set them in the dataframe
    if pre_array.size:
        pre_peak = np.max(pre_array)
        row['pre_peak'] = np.max(pre_array)
    else:
        row['pre_peak'] = 0
    post_peak = np.max(post_array)
    row['post-peak'] = np.max(post_array)
    
    #create iterators and local vars for the pre- and -post peak quarters
    precounter = 0
    postcounter = 0
    pre_qtr = 0
    post_qtr = 0
    
    #iterate through the array to find the number of the column with the pre-peak
    for num in row_array:
        if num == pre_peak:
            pre_qtr = precounter
            break
        else:
            precounter += 1
    #use the number to calculate the column where the pre-peak occurs
    row['pre-peak_qtr'] = 2000.25 + (pre_qtr / 4)
    
    #do the same for the post-peak(in reverse)
    for num in reversed(row_array):
        if num == post_peak:
            post_qtr = 36 - postcounter
        else:
            postcounter += 1
    row['post-peak_qtr'] = 2000.25 + (post_qtr / 4)
    
    #calculate the recovery quarter- the point at which the area has surpassed its pre-nadir job numbers
    recovcounter = 0
    for num in post_array:
        if num >= pre_peak:
            recov_qtr = recovcounter
            break
        else:
            recovcounter += 1
    row['recov_qtr'] = 2000.25 + (recovcounter /4)



In [ ]:
df_empl.columns

## Getting the targets


In [ ]:
# Primary Target: did the area recover at all? Did they reach their former job numbers before the next recession start?
df_empl['recovery'] = (df_empl['post-peak'] >= df_empl['pre-peak']) * 1

#Secondary Targets: Time of decline (between peak and nadir) and time to recover (nadir to recovery)
df_empl['decline_qtrs'] = (df_empl['nadir_qtr'] - df_empl['pre-peak_qtr']) * 4
df_empl['recovery_qtrs'] = (df_empl['recov_qtr'] - df_empl['nadir_qtr']) * 4

#Tertiary Target: job growth. Difference b/w pre-peak and post-peak
df_empl['emp_delta'] = df_empl['post-peak'] - df_empl['pre-peak'] 



In [ ]:
df_empl.head(20)

In [ ]:
df_empl 

In [ ]:
df_empl.head()

In [ ]:
df2 = import_all(recession2_years)

In [ ]:
df2

In [ ]:
df2 = create_df_empl(df2)
df2.columns

In [ ]:
df2 = df2.reset_index()

In [ ]:
df = import_all(recession2_years)
df = df.drop(drop_columns, axis =1)
df = create_df_empl(df)
df = df.dropna(axis = 0)
df = df.reset_index()
df['nadir'] = df.iloc[:,9:].min(axis=1)
df['nadir_qtr'] = df.iloc[:,9:].idxmin(axis=1)
df['nadir_qtr_ct'] = (df['nadir_qtr'] - 2007.25) * 4
df['pre-peak'] = 0
df['post-peak'] = 0
for index, row in df.iterrows():
    slicer = int(row['nadir_qtr_ct'] + 3)
    pre_peak = row.iloc[2:slicer].max()
    df.iloc[index,53] = pre_peak
    post_peak = row.iloc[slicer:].max()
    df.iloc[index,54] = post_peak
df['recovery'] = df['post-peak'] >= df['pre-peak']
df['delta'] = df['post-peak'] - df['pre-peak']

In [ ]:
df


In [ ]:
df.info()